In [1]:
import pandas as pd
import tensorflow as tf
import keras
from keras.models import model_from_json
import h5py
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
import rasterio
from osgeo import gdal, gdal_array

2023-04-24 21:08:02.250460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
cit_sor_model_future = load_model('/Users/maddie/Projects/CPSC_597/deploy_webapp/saved_models/Citharichthys_sordidus_future.h5')

In [6]:
data = [[32.555,-123.44]]
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['deci_lat', 'deci_lon'])
  

In [8]:
inRas=gdal.Open('/Users/maddie/Projects/CPSC_597/data/modified_data/stacked_bio_oracle_future/bio_oracle_future_stacked.tif')
myarray=inRas.ReadAsArray()

len_pd=np.arange(len(df))
lon=df["deci_lon"]
lat=df["deci_lat"]
lon=lon.values
lat=lat.values
    
    
row=[]
col=[]


src=rasterio.open('/Users/maddie/Projects/CPSC_597/data/modified_data/stacked_bio_oracle_future/bio_oracle_future_stacked.tif', crs= 'espg: 4326')
for i in len_pd:
    row_n, col_n = src.index(lon[i], lat[i])# spatial --> image coordinates
    row.append(row_n)
    col.append(col_n)
    
mean_std=pd.read_csv('/Users/maddie/Projects/CPSC_597/data/modified_data/stacked_bio_oracle_future/future_env_bio_mean_std.txt',sep="\t")
mean_std=mean_std.to_numpy()

X=[]
for j in range(0,9):
    print(j)
    band=myarray[j]
    x=[]
        
    for i in range(0,len(row)):
        value= band[row[i],col[i]]
        if value <-1000:
            x.append(value)
        else:
            value = ((value - mean_std.item((j,1))) / mean_std.item((j,2))) # scale values
            x.append(value)
    X.append(x)

X.append(row)
X.append(col)

X =np.array([np.array(xi) for xi in X])
    
df=pd.DataFrame(X)
df=df.T
    
    #drop any rows with no-data values
df=df.dropna(axis=0, how='any')
input_X=df.loc[:,0:8]
    
row=df[9]
col=df[10]
    
row_col=pd.DataFrame({"row":row,"col":col})
   
input_X=input_X.values
    
    #convert rows and col indices back to array
row=row.values
col=col.values
    
prediction_array=np.save('/Users/maddie/Projects/CPSC_597/deploy_webapp/predictions/prediction_array.npy',input_X)
prediction_pandas=row_col.to_csv('/Users/maddie/Projects/CPSC_597/deploy_webapp/predictions/prediction_row_col.csv')

    #predicting outcome
input_X=np.load('/Users/maddie/Projects/CPSC_597/deploy_webapp/predictions/prediction_array.npy')
df=pd.DataFrame(input_X)

    #create copy of band to later subset values in
new_band=myarray[1].copy()
new_band.shape

new_values = cit_sor_model_future.predict(x=input_X,verbose=0) ###predict output value

    ##take the prob. of presence (new_value.item((0,1))) and put into numpy array
new_band_values=[]
   
for i in new_values:
    new_value=i[1]
    new_band_values.append(new_value)
new_band_values=np.array(new_band_values)



0
1
2
3
4
5
6
7
8


In [19]:
my_string = "Predicted Probability: "
resultdf = pd.DataFrame(new_band_values, columns=['result'])
result_value = resultdf['result'].values[0]
result = my_string + str(result_value)